### Домашнее задание

1. Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
3. Повторить п.2, но используя уже не медиану, а max
4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

In [19]:
!pip install razdel

In [20]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 4.3MB/s 
     |████████████████████████████████| 8.2MB 10.7MB/s 


In [21]:
from gensim.corpora.dictionary import Dictionary

#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  
# pip install pymorphy2

import nltk

In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

import matplotlib.pyplot as plt

%matplotlib inline

In [106]:
import itertools
import pandas as pd

In [ ]:
from google.colab import files

In [4]:
# Загрузка файлов с гугл диска
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   39G   70G  36% /
tmpfs            64M     0   64M   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
shm             5.9G     0  5.9G   0% /dev/shm
tmpfs           6.4G   36K  6.4G   1% /var/colab
/dev/sda1        76G   41G   35G  55% /etc/hosts
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive           100G   35G   66G  35% /content/gdrive


In [10]:
!ls /content/gdrive/MyDrive/Colab Notebooks/ML_in_business/les_02

articles.csv	  HW_02		 users_articles.csv
articles_idf.csv  stopwords.txt  users_churn.csv


In [11]:
!cp /content/gdrive/MyDrive/Colab Notebooks/ML_in_business/les_02/articles.csv .

In [12]:
!ls

articles.csv  gdrive  sample_data


In [14]:
news = pd.read_csv("articles.csv")

In [15]:
news.head(3)

,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [16]:
!cp /content/gdrive/MyDrive/'Colab Notebooks'/ML_in_business/les_02/users_articles.csv .

In [17]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


### 1. Получаем векторные представления новостей

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
stopword_ru = stopwords.words('russian')
print(len(stopword_ru))

morph = pymorphy2.MorphAnalyzer()

151


In [26]:
with open('/content/gdrive/MyDrive/Colab Notebooks/ML_in_business/les_02/stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [27]:
stopword_ru[-1]

'thru'

In [28]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub("n", ' ', text)

    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    #print(tokens)
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
                #print(temp_cach)
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
                #print(w,' : ',temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    #print(words_lem_without_stopwords)
    return words_lem_without_stopwords

In [29]:
list(tokenize(news.iloc[0,1][:100],   ))

[Substring(0, 11, 'Заместитель'),
 Substring(12, 24, 'председателя'),
 Substring(24, 25, 'n'),
 Substring(25, 38, 'правительства'),
 Substring(39, 41, 'РФ'),
 Substring(41, 42, 'n'),
 Substring(42, 48, 'Сергей'),
 Substring(48, 49, 'n'),
 Substring(49, 55, 'Иванов'),
 Substring(55, 56, 'n'),
 Substring(56, 62, 'избран'),
 Substring(63, 76, 'председателем'),
 Substring(77, 83, 'совета'),
 Substring(83, 84, 'n'),
 Substring(84, 87, 'ПБК'),
 Substring(88, 92, 'ЦСКА'),
 Substring(92, 93, 'n'),
 Substring(93, 94, '.'),
 Substring(95, 98, 'Как'),
 Substring(99, 100, 'с')]

In [30]:
norm_text = news.iloc[:2,1].apply(lambda x: clean_text(x), 1)
norm_text

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Possible nested set at position 39
  from ipykernel import kernelapp as app


0    заместитель председателя правительства рф серг...
1    матч  финала кубка россии по футболу был приос...
Name: title, dtype: object

In [ ]:
#lemmatization(norm_text[0])

In [32]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

CPU times: user 26.3 s, sys: 1.68 s, total: 27.9 s
Wall time: 28 s


In [33]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: user 4min 13s, sys: 425 ms, total: 4min 13s
Wall time: 4min 14s


А теперь в 3 строчки обучим нашу модель

In [34]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [125]:
N_topic = 30 # число тем - гиперпараметр

In [ ]:
# todo не забыть поменять количество тем

In [126]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=N_topic, id2word=common_dictionary)#, passes=10)

CPU times: user 58.4 s, sys: 29.4 s, total: 1min 27s
Wall time: 55.7 s


In [127]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [65]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(N_topic):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [128]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(N_topic)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(N_topic)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28,topic_29
0,6,0.079679,0.044562,0.0,0.151755,0.014016,0.0,0.0,0.116619,0.0,0.360513,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.078800,0.000000,0.000000,0.132336,0.0,0.000000,0.0,0.0,0.01558,0.000000,0.0,0.0,0.0
1,4896,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.570644,0.0,0.0,0.406592,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0
2,4897,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.040167,0.000000,0.000000,0.330829,0.000000,0.0,0.0,0.402190,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.205148,0.0,0.0,0.0
3,4898,0.000000,0.047681,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.409230,0.033312,0.028951,0.013223,0.000000,0.000000,0.0,0.0,0.438227,0.000000,0.000000,0.000000,0.0,0.020025,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0
4,4899,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.250674,0.000000,0.000000,0.000000,0.000000,0.030061,0.0,0.0,0.084644,0.449656,0.162442,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0


### Следующий шаг - векторные представления пользователей

In [67]:
users.head(10)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"
3,u101138,"[5933, 6186, 5055, 6977, 5206, 488389]"
4,u108248,"[707, 1144, 2532, 2928, 3133, 324592]"
5,u106662,"[323868, 323426, 324267, 322426, 324104, 1550]"
6,u105949,"[293138, 294471, 295012, 294736, 293949, 3544]"
7,u102457,"[6928, 5009, 6940, 7629, 7644, 512736]"
8,u104124,"[322838, 324699, 322991, 322120, 324327, 472331]"
9,u101386,"[7827, 6427, 7394, 7151, 6335, 487254]"


In [129]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(N_topic)]].values))

In [107]:
def get_user_embedding(user_articles_list, metric = np.max): # mean, median, max 
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    #print(user_vector)
    user_vector = metric(user_vector, 0)
    return user_vector

In [48]:
target = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/ML_in_business/les_02/users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [109]:
def get_preds (emb_metric):

    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, emb_metric), 1)])
    user_embeddings.columns = ['topic_{}'.format(i) for i in range(N_topic)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(N_topic)]]
    user_embeddings.head(3)

    X = pd.merge(user_embeddings, target, 'left')
    #разделим данные на train/test
    X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(N_topic)]], 
                                                    X['churn'], random_state=0, test_size=0.25)
    logreg = LogisticRegression(C=1.0)
    #обучим логистич модель
    logreg.fit(X_train, y_train)
    #наши прогнозы для тестовой выборки
    preds = logreg.predict_proba(X_test)[:, 1]
    return preds

In [130]:
preds_max = get_preds (np.max)
preds_mean = get_preds (np.mean)
preds_median = get_preds (np.median)

### Рассчитаем Precision, Recall, F_score

In [115]:
def calc_metrics(y_test, preds):

    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    roc_auc = roc_auc_score(y_test, preds)
    return thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc

In [133]:
preds = [preds_max, preds_mean, preds_median]
metrics = []

for p in preds:
    threshold, fscore, precision, recall, roc_auc = calc_metrics(y_test, p)
    metrics.append([threshold, fscore, precision, recall, roc_auc])

In [132]:
metrics_df = pd.DataFrame(metrics,
                          columns = ['threshold', 'f-score', 'precision', 'recall', 'roc_auc'],
                          index = ['preds_max', 'preds_mean', 'preds_median'])
metrics_df

,threshold,f-score,precision,recall,roc_auc
preds_max,0.371032,0.836066,0.839506,0.832653,0.982041
preds_mean,0.261842,0.711281,0.669065,0.759184,0.955630
preds_median,0.219732,0.706897,0.611940,0.836735,0.955983


In [ ]:
# 20 тем
	        #threshold	f-score	precision	recall	roc_auc
#preds_max	0.361413	0.702703	0.716102	0.689796	0.950316
#preds_mean	0.278266	0.652591	0.615942	0.693878	0.939422
#preds_median	0.273866	0.762082	0.699659	0.836735	0.968242

С увеличением количества тем с 10 до 20 показатели зничительно улучшились, например, roc_auc_ с 0.79 до 0.95

При количестве тем = 20 медиана сработала лучше, вероятно потому, что она более устойчива к выбросам, возможно, какие-то темы с большой вероятностью были выбраны ошибочно, а темы со средней вероятностью лучше характеризуют пользователя

In [ ]:
 # 30 тем
 #           threshold	f-score	precision	recall	roc_auc
#preds_max	0.371032	0.836066	0.839506	0.832653	0.982041
#preds_mean	0.261842	0.711281	0.669065	0.759184	0.955630
#preds_median	0.219732	0.706897	0.611940	0.836735	0.955983

Но при увеличении количества тем до 30, лучшие показатели получились у максимальной темы. видимо, при увеличении кол-ва тем получается более точное попадание в темы, и хорошо срабатывает max, причем по всем метрикам.

Подскажите, будет ли выложено решение, как сделать взвешивая новости по tfidf? Хотелось бы посмотреть.